# Bayesian Optimization of Fast Charging Protocols

This notebook demonstrates the use of the Ax Bayesian Optimization framework to tune a multi-step CC-CV fast charging protocol for a Li-ion battery simulated using a P2D cell model (w/ plating) in PyBaMM. Essentially, this project seeks to answer the question: "What is the charging protocol that maximizes charge in 30 minutes while minimizing aging to the battery?"

---

## Background

We start with a 3-step charging protocol:  
- Step 1: C-rate C1 for 10 minutes 
- Step 2: C-rate C2 for 10 minutes 
- Step 3: C-rate C3 for 10 minutes 

where C1, C2, and C3 are the parameters to be optimized.


The optimization objective is: 

objective = $Q_{30} - \beta*\log(Q_{lost})$

where: 
- $Q_{30}$ is the charge stored in 30 minutes \[Ah\] 
- $Q_{lost}$ is the capacity lost due to lithium plating \[Ah\] 
- $\beta$ is a dimensionless weight that tunes how much the user wants to target high capacity versus low degradation 

This objective is chosen because it satisfies the following criteria:
1. Rewards high capacity
2. Penalizes lithium plating
3. Tunable
4. Stable

**Goal: Obtain optimal charging profiles targeting i) high capacity and ii) low degradation**

## Load Libraries

In [1]:
#PyBamm Library
import pybamm

#Numpy Library
import numpy as np

#Ax Libraries
from ax.api.client import Client
from ax.api.configs import  RangeParameterConfig

---

## Define the Simulation Function

In this section, we define the `run_P2D` function, which simulates a given charging protocol and returns the optimization objective. 

We use the DFN model with reversible plating enabled (based on LG M50 cell parameters) and evaluate performance after 30 minutes of charge. Before the charge cycle, the cell is discharged to it's minimum voltage at 1C.  


In [2]:
def run_P2D(params,beta):
    C1, C2, C3 = params[0], params[1], params[2]
    
    #Define DFN model with lihtium plating
    model = pybamm.lithium_ion.DFN(options={"lithium plating": "reversible"})
    
    #Define CC Experiment
    experiment = pybamm.Experiment([
        "Discharge at 1C until 2.6V",
        "Hold at 2.6V for 10 minutes",
        f"Charge at {C1}C for 10 minutes",
        f"Charge at {C2}C for 10 minutes",
        f"Charge at {C3}C for 10 minutes",
    ])
    pv = pybamm.ParameterValues("OKane2022")
    
    #Run Simulation
    sim = pybamm.Simulation(model, experiment=experiment,parameter_values=pv)
    sim.solve(solver=pybamm.CasadiSolver(mode="safe", dt_max=1))
    
    #Obtain Simulation Observables
    time_sim = sim.solution["Time [min]"].data
    lost_capacity = np.max(sim.solution['Loss of capacity to negative lithium plating [A.h]'].data)
    
    #Calculate Amount of Charge in 30 minutes
    I = sim.solution["Current [A]"].data          #store current data
    t0 = time_sim[np.where(I < 0)[0][0]]     #index first charging point
    charging_mask = (I < 0) & (time_sim <= t0 + 1800) #define 30 minute charging range
    Q30 = np.trapz(-I[charging_mask], time_sim[charging_mask])   #coulomb count for 30 minute charging range
    Q30_Ah = Q30 / 3600     #convert to Ah
    
    objective = Q30_Ah - beta*np.log(lost_capacity)
    
    print({"lost_capacity": lost_capacity, "Q30": Q30_Ah, "objective": objective})
    return {"objective": objective}


---

## Test Simulation with Different Weights

Before optimizing, it's important to test the P2D model to ensure that the simulation returns sensical values and that the weighting factor ($\beta$) reflects the goals of the user (ex: higher capacity vs less plating). Since the parameters of the P2D model are based on an NMC-graphite chemistry (i.e relatively stable so plating shouldn't be very destructive), I will first target a higher capacity. The following ranking is desired: 

Rank 1 (Worst): Low capacity + High degradation \
Rank 2: Low capacity + Low degradation \
Rank 3: High capacity + High degradation \
Rank 4 (Best): High capacity + Low degradation

If a more passive protocol is desired (i.e less degradation is favored), then ranks 2 and 3 could be switched as is done in the 2nd portion of this notebook. 


In [3]:
beta = 0.008 #beta = 0.011 is very balanced but slightly on less degradation side (passive) | beta = 0.008 is slightly on more capacity side (aggressive)
run_P2D([0.25,0.25,0.25],beta)
run_P2D([0.5,0.5,0.5],beta)
run_P2D([2,2,2],beta)
run_P2D([2.5,2,1.5],beta) 

At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0005620043797975666, 'Q30': 0.010416666666666663, 'objective': 0.07028867398560143}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0006279845491386918, 'Q30': 0.020833333333333325, 'objective': 0.07981729329398196}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.4483241250010145, 'Q30': 0.0833333333333333, 'objective': 0.08975124385218475}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.34430032970418356, 'Q30': 0.08333333333333329, 'objective': 0.09186326094158005}


{'objective': 0.09186326094158005}

### Test Results

Two things to note after this initial test:

1. The P2D + plating model is indeed returning values that make physical sense. As the C-rate is doubled from C/4 -> C/2, the amount of charge is doubled but the capacity lost to plating is nearly the same. However, at higher C-rates (ex: 2C) there is significant plating as expected. Finally, when the current starts high but is decreased as the charge progresses (a common strategy used in fast charging), the degree of plating decreases since lithium intercalation is easiest at low SOC. 


2. $\beta$ = 0.008 results in the ranking that I desired, but again this can be readily changed to reflect a more passive strategy that favors less degradation.

---



## Run Bayesian Optimization

Ax is now called to optimize the charge profile (C1, C2, C3). By default, the algorithm uses Gaussian Process (GP) as the surrogate model and Expected Improvement (EI) as the utiliy function. 

### Initialize Algorithm
The following cell initializes the experiment, defines input parameters, and sets the optimization objective. The maximum C-rate is 3C to avoid Vmax from being reached, and the objective is maximized. 


In [ ]:
#1. Initialize Client for Experiment
client = Client()

#2. Define Input Parameter Name, Type, and Bounds/Values
C1 = RangeParameterConfig(name="C1", parameter_type="float", bounds=(0, 3))   #3C is max to prevent Vmax from being reached
C2 = RangeParameterConfig(name="C2", parameter_type="float", bounds=(0, 3))
C3 = RangeParameterConfig(name="C3", parameter_type="float", bounds=(0, 3))

#3. Configure Experiment
client.configure_experiment(
    parameters=[C1,C2,C3],
    # The following arguments are only necessary when saving to the DB
    name="Fast-Charging-Exp",
    description="Optimize fast charging profile-aggressive",
)

#4. Set Optimization Objective
client.configure_optimization(objective="objective")


### Attach Preexisting Trials

Attach the data from the inital simulation test to train the initial surrogate model.

In [ ]:
# Pairs of previously evaluated parameterizations and associated metric readings
preexisting_trials = [
    (
        {"C1": 0.25, "C2": 0.25, "C3": 0.25},
        {"objective": 0.07028867398560143},
    ),
    (
        {"C1": 0.5, "C2": 0.5, "C3": 0.5},
        {"objective": 0.07981729329398196},
    ),
    (
        {"C1": 2.0, "C2": 2.0, "C3": 2.0},
        {"objective": 0.08975124385218475},
    ),
    (
        {"C1": 2.5, "C2": 2.0, "C3": 1.5},
        {"objective": 0.09186326094158005},
    ),

]

for parameters, data in preexisting_trials:
    # Attach the parameterization to the Client as a trial and immediately complete it with the preexisting data
    trial_index = client.attach_trial(parameters=parameters)
    client.complete_trial(trial_index=trial_index, raw_data=data)

### Run Optimization

Every epoch, obtain 3 new charge profile recommendations and test them using the `run_P2D` function for N epochs (N = 20 takes about 5 minutes). 

In [ ]:
N = 35 #number of BO epochs
for i in range(N):
    
    #Get new trials
    trials = client.get_next_trials(max_trials=3)
    
    # Tell Ax the result of those trials
    for trial_index, parameters in trials.items():
        client.complete_trial(trial_index=trial_index, raw_data=run_P2D(list(parameters.values()),beta))

### Analyze Results

Print the best predicted paramters and show the detailed analysis from the BO algorithm

In [ ]:
best_parameters, prediction, index, name = client.get_best_parameterization()
print("Best Parameters:", best_parameters)
print("Prediction (mean, variance):", prediction)

cards = client.compute_analyses(display=True)

---

## Results & Discussion (i)

The most optimal charge profile the BO algorithm returned was:

1. C1 = 3C 
2. C2 = 0C 
3. C3 = 3C 

lost_capacity: 1.2670776248463689 Q30: 0.12499999999999993 objective: 0.12310629467109654 

Given the constraints of the problem, it makes a lot of sense that the most optimal charging profile is to load as much current as possible at first, let the cell rest, and then load as much current as possible later on (i.e charge-rest-charge). The rest period allows the cell to relax which helps ease the diffusion gradients and thus limits lithium plating. A similar strategy has been discussed in the literature called "Pulse-Current Fast Charging" - which has been experimentally shown to maximize capacity while limiting cell degradation due to the inclusion of rest periods. While the strategy recommended by the Bayesian optimization is not really pulse-current (due to the predefined 10 minute intervals), it is remarkable that the algorithm "learned" this strategy completely on it's own. 

Note that due to the stochastic nature of BO, you may or may not reproduce these results immediately, but may take a few tries. The N = 35 was chosen such that this solution is obtained *most* of the time.



---

## Rerun BO with New Weight


I then wanted to test this algorithm for a more passive charging protocol that favors less degradation slightly more than high capacity, so the whole process is reran with a $\beta$ = 0.015. Keep in mind that if less degradation is completely favored ($\beta$>>0.015), then the optimal solution can be trivial (C1=C2=C3=0). For this case, the following rank is desired:

Rank 1 (Worst): Low capacity + High degradation \
Rank 2: High capacity + High degradation \
Rank 3: Low capacity + Low degradation \
Rank 4 (Best): High capacity + Low degradation

### Test new $\beta$ 

In [4]:
#0. Test new beta and obtain pre-training data
beta = 0.015 
run_P2D([0.25,0.25,0.25],beta)
run_P2D([0.5,0.5,0.5],beta)
run_P2D([2,2,2],beta)
run_P2D([2.5,2,1.5],beta) 
run_P2D([3,0,3],beta) 

At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0005620043797975666, 'Q30': 0.010416666666666663, 'objective': 0.12267668038966935}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0006279845491386918, 'Q30': 0.020833333333333325, 'objective': 0.1314282582595495}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


...

At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.34430032970418356, 'Q30': 0.08333333333333329, 'objective': 0.09932694759879598}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 1.2670776248463689, 'Q30': 0.12499999999999993, 'objective': 0.12144930250830607}


{'objective': 0.12144930250830607}

#### Test Results

With the new $\beta$ = 0.015, the new desired ranking was achieved and the previously recommended best charging profile (3C, 0, 3C) is now the 2nd best out of the set tested. 

### Initialize new optimization and add new pre-training data

In [ ]:
#1. Initialize Client for Experiment
client = Client()

#2. Define Input Parameter Name, Type, and Bounds/Values
C1 = RangeParameterConfig(name="C1", parameter_type="float", bounds=(0, 3))   #3C is max to prevent Vmax from being reached
C2 = RangeParameterConfig(name="C2", parameter_type="float", bounds=(0, 3))
C3 = RangeParameterConfig(name="C3", parameter_type="float", bounds=(0, 3))

#3. Configure Experiment
client.configure_experiment(
    parameters=[C1,C2,C3],
    # The following arguments are only necessary when saving to the DB
    name="Fast-Charging-Exp",
    description="Optimize fast charging profile-passive",
)

#4. Set Optimization Objective
client.configure_optimization(objective="objective")

#. Load Preexisting Trials
preexisting_trials = [
    (
        {"C1": 0.25, "C2": 0.25, "C3": 0.25},
        {"objective": 0.12267668038966935},
    ),
    (
        {"C1": 0.5, "C2": 0.5, "C3": 0.5},
        {"objective": 0.1314282582595495},
    ),
    (
        {"C1": 2.0, "C2": 2.0, "C3": 2.0},
        {"objective": 0.09536691555617977},
    ),
    (
        {"C1": 2.5, "C2": 2.0, "C3": 1.5},
        {"objective": 0.09932694759879598},
    ),

]

for parameters, data in preexisting_trials:
    # Attach the parameterization to the Client as a trial and immediately complete it with the preexisting data
    trial_index = client.attach_trial(parameters=parameters)
    client.complete_trial(trial_index=trial_index, raw_data=data)

### Run New Optimization

In [ ]:
N = 35 #number of BO epochs
for i in range(N):
    
    #Get new trials
    trials = client.get_next_trials(max_trials=3)
    
    # Tell Ax the result of those trials
    for trial_index, parameters in trials.items():
        client.complete_trial(trial_index=trial_index, raw_data=run_P2D(list(parameters.values()),beta))

### Analyze New Results

In [ ]:
#Output New Results
best_parameters, prediction, index, name = client.get_best_parameterization()
print("Best Parameters:", best_parameters)
print("Prediction (mean, variance):", prediction)

cards = client.compute_analyses(display=True)

### Results & Discussion (ii)

The new optimal charge profile that prioritizes low degradation over high capacity is:
1. C1 = 0.91C 
2. C2 = 0C 
3. C3 = 0.54C 

Interestingly, this is more or less the same strategy obtained in the previous optimization except now the currents are optimized to minimize degradation from lithium plating. When prioritizing low degradation, the algorithm recommends a profile that is not really fast charging at all (<1C) - which makes sense. 

To show that this is an optimal solution (at least locally), I ran two more profiles at C-rates 5% above and below the recommended solution. Not only was the BO solution better than both (highest capacity amongst lowest degradation), but it appears to be right at the limit that lithium plating starts to become significant - and the algorithm optimized to this limit completely autonomously. Basically, the algorithm returned the charging profile with the maximum capacity before any significant plating began. (Results below)

Although the new solution does indeed seem optimized to low degradation, again keep in mind that this process is stochastic and so it is possible to obtain a better or worse solution after rerunning. This algorithm was ran 3 times to ensure that this solution was indeed the best that could be found. 

In [5]:
soln = np.array([0.9061734071325732,0,0.5354266018520765]) 
run_P2D(soln,beta)
run_P2D(1.05*soln,beta)
run_P2D(0.95*soln,beta)


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0005620043797975666, 'Q30': 0.03003333352051353, 'objective': 0.14229334724351622}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0007101460026075361, 'Q30': 0.0315350001965392, 'objective': 0.14028559977044916}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0005620043797975666, 'Q30': 0.028531666844487838, 'objective': 0.14079168056749053}


{'objective': 0.14079168056749053}

---

## Conclusions & Next Steps

This Bayesian optimization algorithm demonstrates that a 3-step charging protocol can be optimized effectively and the optimal strategy produced by this algorithm is similar to one shown to be sucessful in the literature. While this does show the effectiveness of BO in optimizing fast-charging protocols, careful consideration needs to be given to what weighting factor is used as even small changes can siginficantly affect the optimal solution. 

Potential next steps:
- N-step charging protocol (instead of just 3)
- Time of each step as a parameter (instead of 10 minutes for all)
- Both N-step + t-step as parameters (most general charging profile)
